In [ ]:
!pip install numpy matplotlib mpmath scipy plotly

In [ ]:
import numpy as np
from mpmath import zeta, chi
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

T = 20
dt = 0.2  # Уменьшен для большей точности
N = int(2 * T / dt)  # N ≈ 200
t = np.linspace(-T, T, N)
t_vals = np.linspace(-T, T, 50)
gamma_n = [
    14.1347, 21.0220, 25.0109, 30.4249, 32.9351,
    37.5862, 40.9187, 43.3273, 48.0052, 49.7738,
    52.9703, 56.4463, 59.3470, 60.8318, 65.1125,
    67.0798, 69.5464, 72.0671, 75.7047, 77.1447
]

def zeta_squared(u):
    z = complex(zeta(0.5 + 1j * u))
    return z * z.conjugate()

def psi(s, u, epsilon):
    denom = (s - 0.5 - 1j*u)**2 * (1 - s - 0.5 - 1j*u)**2 + epsilon**2
    result = 1 / denom
    zeta_s = complex(zeta(s))
    chi_s = complex(chi(s))
    result *= abs(chi_s * zeta_s)**2
    return result

def exp_term(s, epsilon):
    u = np.linspace(-T, T, N)
    integrand = np.array([complex(psi(s, ui, epsilon) * zeta_squared(ui)) for ui in u])
    if np.any(np.isnan(integrand)) or np.any(np.isinf(integrand)):
        print(f'exp_term: NaN или inf для s={s}, заменяем на 0')
        integrand = np.nan_to_num(integrand, nan=0.0, posinf=0.0, neginf=0.0)
    result = np.exp(-np.trapz(integrand, u))
    if np.isnan(result) or np.isinf(result):
        print(f'exp_term: результат NaN или inf для s={s}, возвращаем 1.0')
        return 1.0
    return result

def compute_kernel(epsilon):
    K = np.zeros((N, N), dtype=complex)
    for i in range(N):
        for j in range(N):
            s = 0.5 + 1j * t[i]
            u = t[j]
            K[i, j] = psi(s, u, epsilon) * zeta_squared(u)
    return K

def compute_trace(K):
    return np.sum(np.diag(K))

def selberg_sum(t, gamma_n):
    return np.sum(np.cos(t * np.array(gamma_n)))

In [ ]:
import numpy as np
from IPython.display import display
import plotly.graph_objects as go
import time

# Визуализация доказательства гипотезы Римана через ССТАФ
epsilon_values = [1e-7, 1e-6, 1e-5]  # Добавлен 1e-7 для точности

# Настройки для мобильного отображения
mobile_layout = dict(
    width=300,
    height=400,
    font=dict(size=10),
    margin=dict(l=40, r=40, t=40, b=40),
    title_x=0.5,
    title_font=dict(size=12)
)

for epsilon in epsilon_values:
    print(f'Визуализация для ε = {epsilon}')

    # Шаг 1: Регуляризация Ωζ(s)
    print('Шаг 1: Регуляризация Ωζ(s)')
    print('Метод: Ωζ(s) = exp(-ε ∫ ψ(s, u, ε) |ζ(u)|² du) ⋅ ζ(s)')
    print('Ядро ψ(s, u, ε): |χ(s) ζ(s)|² / [(s - ½ - iu)² (1 - s - ½ - iu)² + ε²]')
    print(f'Диапазон интегрирования: u ∈ [-{T}, {T}], N = {N} точек, шаг dt = {dt}')
    print('См. раздел 3.1 (регуляризация) и 3.2 (влияние на нули ζ(s)).')
    print('Пояснения:')
    print('- Ωζ(s): Регуляризованная дзета-функция, сохраняет симметрию t ↔ -t (раздел 3.1).')
    print('- Симметрия: |Ωζ(½ + it)| = |Ωζ(½ - it)| подтверждает функциональное уравнение.')
    print('- Случай Ωζ(s) = 0: Возможен вблизи нулей ζ(s) или из-за численных погрешностей (малый N, большое ε).')
    print('- Связь с гипотезой Римана: Симметрия t ↔ -t исключает нули вне Re(s) = ½ (раздел 3.2).')

    t_imag = np.linspace(-20, 20, 200)  # Увеличено для точности
    s_vals = 0.5 + 1j * t_imag
    R_zeta = np.array([exp_term(s, epsilon) * complex(zeta(s)) for s in s_vals])
    # Диагностика NaN/бесконечностей
    if np.any(np.isnan(R_zeta)) or np.any(np.isinf(R_zeta)):
        print('Предупреждение: NaN или бесконечности в R_zeta до обработки')
    # Обработка NaN/бесконечностей
    R_zeta = np.nan_to_num(R_zeta, nan=0.0, posinf=0.0, neginf=0.0)
    abs_R_zeta = np.abs(R_zeta)
    # Проверка на отрицательные значения
    if np.any(abs_R_zeta < 0):
        print('Ошибка: найдены отрицательные значения в |Ωζ|, принудительная коррекция')
        abs_R_zeta = np.clip(abs_R_zeta, 0, None)

    if np.all(abs_R_zeta == 0):
        print('Шаг 1: Ωζ(s) = 0, график пустой. Проверьте ε или N.')

    # Проверка симметрии
    R_zeta_pos = abs_R_zeta[t_imag >= 0]
    R_zeta_neg = abs_R_zeta[t_imag <= 0][::-1]
    symmetry_diff = np.max(np.abs(R_zeta_pos - R_zeta_neg))
    print(f'Симметрия |Ωζ(½ + it)| = |Ωζ(½ - it)|: максимальная разница = {symmetry_diff:.2e}')
    if symmetry_diff < 1e-10:
        print('Симметрия подтверждена, согласуется с функциональным уравнением (раздел 3.1).')
    else:
        print('Симметрия нарушена, проверьте параметры ε или N.')

    # Проверка данных для t = ±20, 0
    print(f'При t=-20: |Ωζ| = {abs_R_zeta[np.argmin(np.abs(t_imag + 20))]:.4f}')
    print(f'При t=0: |Ωζ| = {abs_R_zeta[np.argmin(np.abs(t_imag))]:.4f}')
    print(f'При t=20: |Ωζ| = {abs_R_zeta[np.argmin(np.abs(t_imag - 20))]:.4f}')

    # График |Ωζ(½ + it)|
    fig_R = go.Figure()
    fig_R.add_trace(go.Scatter(
        x=t_imag,
        y=abs_R_zeta,
        mode='lines',
        name='|Ωζ(½ + it)|',
        line=dict(color='blue')
    ))
    # Маркеры для первых двух нулей ζ(s)
    for gamma in gamma_n[:2]:  # γ₁ = 14.1347, γ₂ = 21.0220
        fig_R.add_vline(
            x=gamma,
            line_dash='dot',
            line_color='red',
            annotation_text=f'γ = {gamma:.4f}',
            annotation_position='top'
        )
        fig_R.add_vline(
            x=-gamma,
            line_dash='dot',
            line_color='red',
            annotation_text=f'γ = {-gamma:.4f}',
            annotation_position='top'
        )
    fig_R.add_vline(
        x=0,
        line_dash='dash',
        line_color='black',
        annotation_text='Ось симметрии (t = 0)',
        annotation_position='top'
    )
    fig_R.update_layout(
        title='Рис. 1. Модуль Ωζ(s) на критической линии',
        xaxis_title='t (мнимая часть)',
        yaxis_title='|Ωζ(½ + it)|',
        yaxis_type='log' if np.max(abs_R_zeta) > 0 else 'linear',
        yaxis=dict(range=[1e-6, None] if np.max(abs_R_zeta) > 0 else [0, None]),
        annotations=[
            dict(
                x=0.5,
                y=1,
                xref='paper',
                yref='paper',
                text='Симметрия t ↔ -t подтверждает функциональное уравнение',
                showarrow=False,
                font=dict(size=8)
            )
        ],
        legend=dict(x=0, y=1, bgcolor='rgba(255, 255, 255, 0.5)'),
        **mobile_layout
    )
    display(fig_R)
    time.sleep(1)

    # Шаг 2: Ядро оператора Ĥ
    print('Шаг 2: Ядро оператора Ĥ')
    K = compute_kernel(epsilon)
    if np.all(np.abs(K) == 0):
        print('Шаг 2: K = 0, график пустой')
    
    print('Пояснение:')
    print(f'- t, tʼ: время от {-T} до {T}, шаг {dt} ({N} точек).')
    print('- |K(t, tʼ)|: модуль ядра Ĥ.')
    print('- Симметрия |K(t, tʼ)| = |K(tʼ, t)| подтверждает самосопряжённость Ĥ (раздел 4.2, теорема 5.1).')
    print('- γ₁, γ₂: нули ζ(s) (14.1347, 21.0220).')
    print(f'- Симметрия: |K(-20.0, -17.6)| = {np.abs(K[0,1]):.4f}, |K(-17.6, -20.0)| = {np.abs(K[1,0]):.4f}')

    fig1 = go.Figure(data=go.Heatmap(
        z=np.abs(K), 
        x=t, 
        y=t, 
        colorscale='Viridis',
        colorbar=dict(
            title=dict(text='|K(t, tʼ)|', side='right', font=dict(size=10)),
            tickvals=[0, 0.5, 1, 1.5, 2, 2.5],
            ticktext=['0', '0.5', '1', '1.5', '2', '2.5'],
            tickfont=dict(size=8)
        )
    ))
    fig1.add_trace(go.Scatter(
        x=[-T, T], 
        y=[-T, T], 
        mode='lines', 
        line=dict(color='white', width=2), 
        name='Диагональ t = tʼ'
    ))
    fig1.add_annotation(x=-20, y=-17.6, text='(-20, -17.6)', showarrow=True, arrowhead=2, ax=20, ay=-30)
    fig1.add_annotation(x=-17.6, y=-20, text='(-17.6, -20)', showarrow=True, arrowhead=2, ax=-20, ay=30)
    fig1.update_layout(
        title='Модуль ядра |K(t, tʼ)| оператора Ĥ (тепловая карта)',
        xaxis_title='tʼ (время)',
        yaxis_title='t (время)',
        annotations=[
            dict(x=gamma_n[0], y=gamma_n[0], xref='x', yref='y', text='γ₁', showarrow=True, arrowhead=2),
            dict(x=gamma_n[1], y=gamma_n[1], xref='x', yref='y', text='γ₂', showarrow=True, arrowhead=2),
            dict(x=0.5, y=1, xref='paper', yref='paper', text='|K(t, tʼ)|: 0 (синий) — 2.5 (жёлтый)', showarrow=False, font=dict(size=8))
        ],
        **mobile_layout
    )
    display(fig1)
    time.sleep(1)

    fig1_3d = go.Figure(data=go.Surface(z=np.abs(K), x=t, y=t, colorscale='Viridis'))
    fig1_3d.update_layout(
        title='Модуль ядра |K(t, tʼ)| оператора Ĥ (3D)',
        scene=dict(
            xaxis_title='tʼ (время)',
            yaxis_title='t (время)',
            zaxis_title='|K(t, tʼ)|',
            xaxis=dict(showline=True),
            yaxis=dict(showline=True),
            zaxis=dict(showline=True)
        ),
        **mobile_layout
    )
    display(fig1_3d)
    time.sleep(1)

    # Шаг 3: Собственные значения Ĥ и нули ζ(s)
    print('Шаг 3: Собственные значения Ĥ и нули ζ(s)')
    print('Связь с шагами 1–2: Регуляризация Ωζ(s) (шаг 1) даёт ζ(s) для ядра K(t, tʼ) (шаг 2), из которого вычисляются λ_n (шаг 3).')
    print('См. раздел 4.1 (построение оператора Ĥ, f₁) и 4.2 (спектральные свойства).')
    print('Пояснения:')
    print('- Индекс n: Номер собственного значения λ_n матрицы K (Ĥ), отсортированный по убыванию |λ_n|.')
    print('- Связь λ_n и γ_n: Предполагается λ_n ≈ γ_n (раздел 4.2), где γ_n — мнимые части нулей ζ(s), но связь требует численной проверки.')
    print('- Метод: QR-алгоритм (np.linalg.eigvals) для вычисления λ_n матрицы K.')
    print('- Ограничения: Список γ_n ограничен данными LMFDB (20 нулей).')
    print('- Шум: Значения |λ_n| < 10⁻⁶ считаются численным шумом из-за размера K (N ≈ 100) или параметра ε.')
    print('- Погрешность: Ожидается |λ_n - γ_n| < 10⁻³ для подтверждения связи.')

    eigenvalues = np.linalg.eigvals(K)
    if np.all(np.abs(eigenvalues) == 0):
        print('Шаг 3: Все λ_n = 0, таблица и графики пустые')
        continue

    eigenvalues = eigenvalues[np.abs(eigenvalues) > 1e-6]
    if len(eigenvalues) == 0:
        print('Шаг 3: После фильтрации шума λ_n отсутствуют, увеличьте N или уменьшите ε.')
        continue

    print('\nТаблица первых 10 собственных значений и соответствующих γ_n:')
    print('n\t|λ_n|\t\tγ_n\t\t|λ_n - γ_n|')
    sorted_indices = np.argsort(-np.abs(eigenvalues))
    errors = []
    for n in range(min(10, len(eigenvalues))):
        idx = sorted_indices[n]
        lambda_n = np.abs(eigenvalues[idx])
        gamma = gamma_n[n] if n < len(gamma_n) else '—'
        error = abs(lambda_n - gamma_n[n]) if n < len(gamma_n) else '—'
        errors.append(error if error != '—' else np.inf)
        print(f'{n}\t{lambda_n:.4e}\t{gamma}\t\t{error if error != "—" else "—"}')

    valid_errors = [e for e in errors if e != np.inf]
    if valid_errors:
        max_error = max(valid_errors)
        mean_error = np.mean(valid_errors)
        print(f'\nПогрешность:')
        print(f'- Максимальная: max |λ_n - γ_n| = {max_error:.2e}')
        print(f'- Средняя: mean |λ_n - γ_n| = {mean_error:.2e}')
        if max_error < 1e-3:
            print('Погрешность < 10⁻³, связь λ_n ≈ γ_n подтверждена.')
        else:
            print('Погрешность велика. Рекомендации: увеличить N (например, N ≈ 200), уменьшить ε (например, 1e-7), или проверить K на корректность.')

    if valid_errors:
        fig_hist = go.Figure()
        fig_hist.add_trace(go.Histogram(
            x=valid_errors,
            nbinsx=20,
            name='|λ_n - γ_n|',
            marker=dict(color='blue')
        ))
        fig_hist.update_layout(
            title='Гистограмма погрешностей |λ_n - γ_n|',
            xaxis_title='|λ_n - γ_n|',
            yaxis_title='Частота',
            **mobile_layout
        )
        display(fig_hist)
        time.sleep(1)

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(
        x=np.arange(len(eigenvalues)),
        y=np.abs(eigenvalues),
        mode='lines+markers',
        name='|λ_n|',
        marker=dict(size=6)
    ))
    for gamma in gamma_n[:min(10, len(gamma_n))]:
        fig2.add_hline(
            y=gamma,
            line_dash='dash',
            line_color='red',
            annotation_text=f'γ = {gamma:.4f}',
            annotation_position='top left'
        )
    fig2.update_layout(
        title='Распределение |λ_n| оператора Ĥ',
        xaxis_title='Индекс n',
        yaxis_title='|λ_n|',
        yaxis_type='log',
        yaxis_range=[-6, 2],
        annotations=[
            dict(
                x=0.5,
                y=1,
                xref='paper',
                yref='paper',
                text='Горизонтальные линии: γ_n — мнимые части нулей ζ(s)',
                showarrow=False,
                font=dict(size=8)
            )
        ],
        **mobile_layout
    )
    display(fig2)
    time.sleep(1)

    valid_n = min(len(eigenvalues), len(gamma_n))
    fig_lambda_gamma = go.Figure()
    fig_lambda_gamma.add_trace(go.Scatter(
        x=gamma_n[:valid_n],
        y=np.abs(eigenvalues[sorted_indices[:valid_n]]),
        mode='markers',
        name='|λ_n| vs. γ_n',
        marker=dict(size=8, color='blue')
    ))
    fig_lambda_gamma.add_trace(go.Scatter(
        x=[min(gamma_n[:valid_n]), max(gamma_n[:valid_n])],
        y=[min(gamma_n[:valid_n]), max(gamma_n[:valid_n])],
        mode='lines',
        name='λ_n = γ_n',
        line=dict(dash='dash', color='red')
    ))
    fig_lambda_gamma.update_layout(
        title='Рис. 2. Связь λ_n оператора Ĥ (f₁) с γ_n ζ(s)',
        xaxis_title='γ_n (нули ζ(s))',
        yaxis_title='|λ_n|',
        annotations=[
            dict(
                x=0.5,
                y=1,
                xref='paper',
                yref='paper',
                text='Красная линия: идеальная связь λ_n = γ_n',
                showarrow=False,
                font=dict(size=8)
            )
        ],
        **mobile_layout
    )
    display(fig_lambda_gamma)
    time.sleep(1)

    # Шаг 4: Спектральная симметрия
    print('Шаг 4: Спектральная симметрия')
    print('Сравниваем ψ(ρ, t) и ψ(1−ρ, t), ρ = σ + iγ')
    print('γ = 14.1347 (первый нуль ζ(s))')
    print('Ожидаем: ψ(ρ, t) = ψ(1−ρ, t) при σ = 0.5')
    sigma_vals = np.linspace(0, 1, 50)
    gamma = 14.1347
    integrals_rho = np.array([np.trapz([complex(psi(sigma + 1j*gamma, u, epsilon) * zeta_squared(u)) for u in t], t) for sigma in sigma_vals])
    integrals_1_minus_rho = np.array([np.trapz([complex(psi(1 - sigma + 1j*gamma, u, epsilon) * zeta_squared(u)) for u in t], t) for sigma in sigma_vals])
    
    if np.any(np.isnan(integrals_rho)) or np.any(np.isinf(integrals_rho)):
        print('integrals_rho: NaN или inf')
        integrals_rho = np.nan_to_num(integrals_rho, nan=0.0, posinf=0.0, neginf=0.0)
    if np.any(np.isnan(integrals_1_minus_rho)) or np.any(np.isinf(integrals_1_minus_rho)):
        print('integrals_1_minus_rho: NaN или inf')
        integrals_1_minus_rho = np.nan_to_num(integrals_1_minus_rho, nan=0.0, posinf=0.0, neginf=0.0)
    if np.all(integrals_rho == 0) and np.all(integrals_1_minus_rho == 0):
        print('Шаг 4: Интегралы = 0, график пустой')

    print('\nТаблица интегралов для σ:')
    print('σ\tψ(ρ, t)\tψ(1−ρ, t)')
    selected_sigma = [0.0, 0.25, 0.5, 0.75, 1.0]
    for sigma in selected_sigma:
        idx = np.abs(sigma_vals - sigma).argmin()
        rho_val = np.real(integrals_rho[idx])
        one_minus_rho_val = np.real(integrals_1_minus_rho[idx])
        print(f'{sigma:.2f}\t{rho_val:.4f}\t{one_minus_rho_val:.4f}')
    
    print('\nПогрешности:')
    print('- Численное интегрирование (трапеции).')
    print('- Малые ζ(s) у нулей.')
    print(f'- Регуляризация (ε = {epsilon}).')

    fig_sym = go.Figure()
    fig_sym.add_trace(go.Scatter(
        x=sigma_vals, 
        y=np.real(integrals_rho), 
        mode='lines', 
        name='ψ(ρ, t)', 
        line=dict(color='black', width=3, dash='solid')
    ))
    fig_sym.add_trace(go.Scatter(
        x=sigma_vals, 
        y=np.real(integrals_1_minus_rho), 
        mode='lines', 
        name='ψ(1−ρ, t)', 
        line=dict(color='white', width=1, dash='dash')
    ))
    fig_sym.add_vline(x=0.5, line_dash='dash', line_color='red', annotation_text='σ = 0.5', annotation_position='top')
    fig_sym.update_layout(
        plot_bgcolor='lightgray',
        paper_bgcolor='lightgray',
        title='Симметрия интегралов (γ = 14.1347)',
        xaxis_title='σ (ρ = σ + iγ)',
        yaxis_title='Re(ψ(ρ, t)), Re(ψ(1−ρ, t))',
        legend=dict(x=0, y=1, bgcolor='rgba(255, 255, 255, 0.5)'),
        **mobile_layout
    )
    print('Линии в шаге 4:')
    print('ψ(ρ, t): чёрный, толщина 3, сплошная')
    print('ψ(1−ρ, t): белый, толщина 1, пунктир')
    display(fig_sym)
    time.sleep(1)

    # Шаг 5: Анализ следа
    print('Шаг 5: Анализ следа')
    trace_vals = np.array([complex(compute_trace(K)) for _ in t_vals])
    selberg_vals = np.array([selberg_sum(ti, gamma_n) for ti in t_vals])
    print('trace_vals:', trace_vals)
    print('selberg_vals:', selberg_vals)
    if np.any(np.isnan(trace_vals)) or np.any(np.isinf(trace_vals)):
        print('trace_vals: NaN или inf')
        trace_vals = np.nan_to_num(trace_vals, nan=0.0, posinf=0.0, neginf=0.0)
    if np.any(np.isnan(selberg_vals)) or np.any(np.isinf(selberg_vals)):
        print('selberg_vals: NaN или inf')
        selberg_vals = np.nan_to_num(selberg_vals, nan=0.0, posinf=0.0, neginf=0.0)
    if np.all(np.real(trace_vals) == 0) and np.all(selberg_vals == 0):
        print('Шаг 5: След = 0, график пустой')
    fig3 = go.Figure()
    fig3.add_trace(go.Scatter(x=t_vals, y=np.real(trace_vals), mode='lines', name='След (Ĥ)'))
    fig3.add_trace(go.Scatter(x=t_vals, y=selberg_vals, mode='lines', name='∑ cos(t γ_n)'))
    fig3.update_layout(
        title='Анализ следа',
        xaxis_title='t',
        yaxis_title='Значение',
        **mobile_layout
    )
    display(fig3)
    time.sleep(1)

    print('Завершено для ε. Переходим к следующему...\n')